# Mass flow conservation

In [ ]:
import aerokit.aero.degree     as deg
import aerokit.aero.Isentropic as Is
import aerokit.aero.ShockWave  as sw
import numpy                 as np
import matplotlib.pyplot     as plt

fontsize=14 ; plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = fontsize 
plt.rcParams['lines.linewidth'] = 1.5
#plt.rcParams.update({'font.size': fontsize})

npoints = 200
gam     = 1.4

fig=plt.figure(1, figsize=(16,12))
fig.suptitle('Polar of Shock-Waves, $\gamma = %.1f$'%gam, fontsize=15, y=0.90)

macharray = [ 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2., 2.2, 2.4, 2.6, 3., 3.5, 4., 5., 10., 100. ]

for m in macharray:
    sig = np.linspace(deg.asin(1./m), 90., npoints+1)
    dev = sw.deflection_Mach_sigma(m, sig, gam)
    plt.plot(dev, sig, 'k-')
    ilab = int(npoints/3)
    figratio = .5
    ang  = deg.atan((sig[ilab+1]-sig[ilab])/(dev[ilab+1]-dev[ilab])*figratio)
    plt.text(dev[ilab], sig[ilab], '%.3g'%m, horizontalalignment='left', verticalalignment='top',
             fontsize=8, bbox=dict(edgecolor='grey', facecolor='white', alpha=0.5),
             rotation=ang)


mach=np.logspace(np.log10(1.01), np.log10(50), npoints+1, base=10)

# --- plot maximum deviation line ---
sig = sw.sigma_DevMax(mach, gam)
dev = sw.deflection_Mach_sigma(mach, sig, gam)
plt.plot(dev,sig)
# --- plot sonic line ---
sig = sw.sigma_Sonic(mach, gam)
dev = sw.deflection_Mach_sigma(mach, sig, gam)
plt.plot(dev,sig, 'b--')

    #labels.append(legends[i]+", t=%.1f"%results[i][t].time)
#legend(labels, loc='upper left',prop={'size':10})  
plt.axis([0., 50., 0., 90.])
plt.xlabel('deviation $\Delta\\theta$', fontsize=14)
plt.ylabel('shock angle $\sigma$', fontsize=14)
plt.minorticks_on()
plt.grid(which='major', linestyle='-', alpha=0.8)
plt.grid(which='minor', linestyle=':', alpha=0.5)